In [1]:
import matplotlib.pyplot as plt
import matplotlib
import mne
import pandas as pd
import numpy as np

from utils import *

In [2]:
#import BDF and setting important parameters
raw = mne.io.read_raw_bdf('../test_data/error_trigtest2.bdf')


fs = raw.info['sfreq']
#raw.info.ch_names

raw = raw.copy().crop(tmin = 0, tmax = 60)

Extracting EDF parameters from /Users/cindyzhang/Documents/M2/Audiomotor_Piano/AM-EEG/test_data/error_trigtest2.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...


In [3]:
#loading the raw events
#the stim channel is called 'Status' 
events = mne.find_events(raw, stim_channel='Status', shortest_event=1) #raises exception if shortest event is default 2...?

Trigger channel has a non-zero initial value of 130816 (consider using initial_event=True to detect this event)
1964 events found
Event IDs: [65282 65284 65288 65290 65296]


In [7]:
#sorting events into different IDs

t_keystrokes = clean_triggers(events[events[:,2]==65282])
t_inv = clean_triggers(events[events[:,2]==65284])
t_shinv = clean_triggers(events[events[:,2]==65288])
t_norm =  clean_triggers(events[events[:,2]==65296])
t_mode = clean_triggers(events[(events[:,2]!=65282) & (events[:,2]!=65290)]) #all triggers that indicate mode switches (i.e. everything except keystrokes)


1. get the start and end times of each phase (array of start and endtimes)
2. see if the time of each keystroke fits in to one of the times

In [12]:
inv_times = find_sections(raw, t_inv, t_mode)
shinv_times = find_sections(raw, t_shinv, t_mode)
norm_times = find_sections(raw, t_norm, t_mode)

In [14]:
def find_keystrokes(raw, t_keystrokes, conditions):
    keystrokes = t_keystrokes[:,0]
    filt_keystrokes = []
    for segment in conditions:
        start = segment[0]
        end = segment[1]

        condition = (keystrokes >= start) & (keystrokes <= end)
        filt_keystrokes.extend(keystrokes[condition])

    indices = np.where(np.isin(t_keystrokes[:, 0], filt_keystrokes))[0]
    filt_keystrokes_events = t_keystrokes[indices]
    return filt_keystrokes_events


find_keystrokes(raw, t_keystrokes, norm_times)

array([[ 35736,  65280,  65282],
       [ 40836,  65280,  65282],
       [ 42917,  65280,  65282],
       [ 44745,  65280,  65282],
       [ 46776,  65280,  65282],
       [ 51809,  65280,  65282],
       [ 54091,  65280,  65282],
       [ 58967,  65280,  65282],
       [ 63283,  65280,  65282],
       [ 65665,  65280,  65282],
       [ 67872,  65280,  65282],
       [ 69906,  65280,  65282],
       [ 73987,  65280,  65282],
       [ 80674,  65280,  65282],
       [ 85184,  65280,  65282],
       [ 87239,  65280,  65282],
       [ 90158,  65280,  65282],
       [ 92722,  65280,  65282],
       [ 97592,  65280,  65282],
       [103941,  65280,  65282],
       [107163,  65280,  65282],
       [109150,  65280,  65282]])

In [5]:
""" not good because border effect; better to sort keystrokes directly according to conditions"""

#gathering the segments depending on the key mapping condition
shinv_raw = concat_uniform(raw, t_shinv, 2, fs)
inv_raw = concat_nonuniform(raw, t_inv, t_shinv, fs)

In [6]:
#epoch by keystroke